In [1]:
#r "../../../.nuget/packages/fsharp.systemtextjson/0.12.12/lib/netstandard2.0/FSharp.SystemTextJson.dll"

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
#r "../src/bin/Debug/netstandard2.1/HafasJsonRpcClient.dll"

In [3]:
open HafasLibrary

Start the node process.

In [5]:
let client = startClient (HafasLibrary.Profile.Db, { defaultClientOptions with
               script = "../../hafas-jsonrpc-server/build/index.js" })

Load the uic identifiers.

In [6]:
let maybeFrom = getIdOfFirstStop (getLocations client "Bielefeld" None)

In [7]:
let maybeTo = getIdOfFirstStop (getLocations client "Berlin" None)

Load the journeys.

In [16]:
let journeysOptions =
    { defaultJourneysOptions with
          results = Some 5
          stopovers = Some true}

In [17]:
let maybeJourneys =
    match maybeFrom, maybeTo with
    | Some from, Some ``to`` ->
        match (getJourneys client from ``to`` (Some journeysOptions)) with
        | Some result -> result.journeys
        | _ -> None
    | _ -> None

Extract journey summaries.

In [18]:
match maybeJourneys with
| Some journeys ->
    journeys
    |> Array.map getJourneySummary
    |> Array.choose id
| _ -> Array.empty

index,origin,departure,destination,arrival
0,Bielefeld Hbf,2020-09-05T11:38:00+02:00,Berlin Hbf (tief),2020-09-05T14:10:00+02:00
1,Bielefeld Hbf,2020-09-05T12:38:00+02:00,Berlin Hbf (tief),2020-09-05T15:05:00+02:00
2,Bielefeld Hbf,2020-09-05T13:20:00+02:00,Berlin Hbf (tief),2020-09-05T16:10:00+02:00
3,Bielefeld Hbf,2020-09-05T13:38:00+02:00,Berlin Hbf (tief),2020-09-05T16:10:00+02:00
4,Bielefeld Hbf,2020-09-05T14:38:00+02:00,Berlin Hbf (tief),2020-09-05T17:06:00+02:00


In [19]:
let maybeFirstJourney =
    match maybeJourneys with
    | Some journeys -> if journeys.Length > 0 then Some journeys.[0] else None
    | _ -> None

In [20]:
let journeyLocations =
    match maybeFirstJourney with
    | Some journey -> getJourneyLocations journey
    | None -> Array.empty

In [22]:
let uri = BRouter.getUri journeyLocations

In [27]:
open Html

In [28]:
let router = iframe [_src uri; _width "100%"; _height "350"] []

In [29]:
display(router)